In [ ]:
# Install required libraries
!pip install pandas requests beautifulsoup4 lxml matplotlib seaborn

import pandas as pd
import requests
from io import StringIO
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os

sns.set(style="whitegrid")

# URLs must be defined before running
presidents_url = "https://en.wikipedia.org/wiki/President_of_the_Philippines"
vps_url = "https://en.wikipedia.org/wiki/Vice_President_of_the_Philippines"

headers = {
    "User-Agent": "Mozilla/5.0"
}

def scrape_all_tables(url, headers):
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        raise Exception(f"Failed to fetch {url} (status {response.status_code})")

    tables = pd.read_html(StringIO(response.text))
    print(f"Found {len(tables)} tables on {url}")
    return tables

tables_pres = scrape_all_tables(presidents_url, headers)
tables_vp = scrape_all_tables(vps_url, headers)

# Inspect tables to find the correct one
for i, t in enumerate(tables_pres):
    print(f"\nTable {i} preview (columns: {t.columns.tolist()}):")
    print(t.head(3))

df_pres = tables_pres[1]
df_vp = tables_vp[1]

def clean_wiki_table_foolproof(df, role_hint):
    """
    Cleans Wikipedia leader tables robustly.
    Detects Name, Term, Party (optional), Start/End Year.
    """

    # Detect name column
    name_col = None
    for c in df.columns:
        if df[c].dtype == object and df[c].astype(str).str.contains(r"[A-Za-z]").any():
            name_col = c
            break

    if not name_col:
        raise Exception(f"Could not detect Name column for role_hint: {role_hint}")

    # Detect term column
    term_col = None
    for c in df.columns:
        if df[c].astype(str).str.contains(r"\d{4}").any():
            term_col = c
            break

    if not term_col:
        raise Exception("Could not detect Term column")

    # Detect party column (optional)
    party_col = None
    for c in df.columns:
        if "party" in str(c).lower() or "political" in str(c).lower():
            party_col = c
            break

    number_col = df.columns[0]

    if party_col:
        df_clean = df[[number_col, name_col, term_col, party_col]].copy()
        df_clean.columns = ["Number", "Name", "Term", "Party"]
    else:
        df_clean = df[[number_col, name_col, term_col]].copy()
        df_clean.columns = ["Number", "Name", "Term"]
        df_clean["Party"] = None

    # Remove footnotes
    df_clean["Name"] = df_clean["Name"].str.replace(r"\[.*?\]", "", regex=True)

    # Parse start and end years
    def parse_term(term):
        years = re.findall(r"\d{4}", str(term))
        if len(years) == 2:
            return pd.Series([int(years[0]), int(years[1])])
        if len(years) == 1:
            return pd.Series([int(years[0]), int(years[0])])
        return pd.Series([None, None])

    df_clean[["Start_Year", "End_Year"]] = df_clean["Term"].apply(parse_term)
    df_clean = df_clean[df_clean["Start_Year"].notnull()].reset_index(drop=True)

    return df_clean

df_pres_clean = clean_wiki_table_foolproof(df_pres, "President")
df_vp_clean = clean_wiki_table_foolproof(df_vp, "Vice President")

print("Presidents Table Preview")
print(df_pres_clean.head())

print("\nVice Presidents Table Preview")
print(df_vp_clean.head())

df_pres_clean.to_csv("philippine_presidents.csv", index=False)
df_vp_clean.to_csv("philippine_vice_presidents.csv", index=False)

print("CSV files saved in")
print(os.getcwd())


Found 18 tables on https://en.wikipedia.org/wiki/President_of_the_Philippines
Found 7 tables on https://en.wikipedia.org/wiki/Vice_President_of_the_Philippines

Table 0 preview (columns: ['President of the Philippines', 'President of the Philippines.1']):
  President of the Philippines President of the Philippines.1
0         Pangulo ng Pilipinas           Pangulo ng Pilipinas
1            Presidential seal              Presidential seal
2        Presidential standard          Presidential standard

Table 1 preview (columns: ['Politics of the Philippines']):
                         Politics of the Philippines
0  Government Constitution of the Philippines Cha...
1  Executive President of the Philippines Bongbon...
2  Legislature Congress of the Philippines 20th C...

Table 2 preview (columns: ['No.', 'Office', 'Incumbent']):
   No.                                   Office     Incumbent
0    1                           Vice President  Sara Duterte
1    2                  President of th